In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder


%matplotlib inline
import keras

#nltk
from nltk.tokenize import sent_tokenize

from tensorflow.keras import layers,Sequential,optimizers,preprocessing

Using TensorFlow backend.


In [2]:
data = pd.read_csv("bbc-text.csv")
data.head(10)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
5,politics,howard hits back at mongrel jibe michael howar...
6,politics,blair prepares to name poll date tony blair is...
7,sport,henman hopes ended in dubai third seed tim hen...
8,sport,wilkinson fit to face edinburgh england captai...
9,entertainment,last star wars not for children the sixth an...


### Lets Expand the text to see news detaily

In [3]:
# # pandas columny ayarları stackoverflow'da soruldu
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

### its not suitable with 100 columns text let's reset te option

In [4]:
pd.reset_option('^display.', silent=True)

In [5]:
data.head(5)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [6]:
data["category"].value_counts(ascending = True)

entertainment    386
tech             401
politics         417
business         510
sport            511
Name: category, dtype: int64

## preparing the data
<p> split the data as train and test
<p> tokenize the words as a sentence
<p> kategorik olan verileri one-hot encoding yapalm label encode
<p> one hot
    
  

In [7]:
train_category , test_category = train_test_split(data["category"] , test_size = 0.2 , random_state = 42)
train_text , test_text = train_test_split(data["text"] , test_size = 0.2 , random_state = 42)

In [8]:
print(train_category.head(3))
print(train_text.head(3))

1490    entertainment
2001         business
1572         business
Name: category, dtype: object
1490    farrell due to make us tv debut actor colin fa...
2001    china continues rapid growth china s economy h...
1572    ebbers  aware  of worldcom fraud former worldc...
Name: text, dtype: object


In [9]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words = 1000 , char_level = False)


### Fit on text from doc.

Updates internal vocabulary based on a list of texts.

In the case where texts contains lists, we assume each entry of the lists to be a token.

Required before using texts_to_sequences or texts_to_matrix.

In [10]:
tokenizer.fit_on_texts(train_text)
tokenizer.fit_on_texts(test_text)

In [11]:
x_train = tokenizer.texts_to_matrix(train_text)
x_test = tokenizer.texts_to_matrix(test_text)


x_train


array([[0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [12]:
### ok bütün sentenceleri sequence vector seklinde gosterdik labellar kaldı laballeri catagoric olduğu için one hot encoder kullanıcam 

le = LabelEncoder()

le.fit(train_category)
y_train = le.transform(train_category)
y_test = le.transform(test_category)

y_train

array([1, 0, 0, ..., 4, 3, 4])

In [13]:
num_classes =np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train , num_classes)
y_test = keras.utils.to_categorical(y_test , num_classes)


In [14]:
y_train

array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [15]:
y_test

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [16]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1780, 1000)
(445, 1000)
(1780, 5)
(445, 5)


### Train the model

In [17]:
batch_size = 32
epochs = 2
drop_ratio = 0.5

In [18]:
# Build the model
model =Sequential()
model.add(layers.Dense(512, input_shape=(1000,)))
model.add(layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [19]:
history = model.fit(x_train , y_train ,
                   batch_size = batch_size,
                   epochs = epochs,
                   validation_split=0.1)

Train on 1602 samples, validate on 178 samples
Epoch 1/2
1602/1602 [==============================] - 1s 648us/sample - loss: 0.4766 - accuracy: 0.8577 - val_loss: 0.1090 - val_accuracy: 0.9663
Epoch 2/2
1602/1602 [==============================] - 0s 153us/sample - loss: 0.0458 - accuracy: 0.9913 - val_loss: 0.0999 - val_accuracy: 0.9663


array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [20]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

445/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Test loss: 0.17377024748352138
Test accuracy: 0.93707865


In [65]:
samples = ["cristiano ronaldo was mad because of the last game he could not score"]

In [66]:
text_labels = le.classes_ 

In [67]:
tokenizer.fit_on_texts(samples)

In [71]:
sample_test = tokenizer.texts_to_matrix(samples)


In [73]:
prediction = model.predict(np.array(sample_test))

print(text_labels)
print(prediction)

['business' 'entertainment' 'politics' 'sport' 'tech']
[[0.16417216 0.19560958 0.17328903 0.28369063 0.18323858]]


In [70]:
predicted_label = text_labels[np.argmax(prediction)]
predicted_label


'sport'